In [2]:
import os
import json
import numpy as np

import torch
import torch.nn as nn
from torch.optim import AdamW
import cv2
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.models import resnet18
import torchvision
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor, Lambda

from glob import glob
import shutil
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! unzip ./drive/MyDrive/poly.zip -d ./zip_data


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./zip_data/polyp/720401.jpg  
  inflating: ./zip_data/polyp/72050.jpg  
  inflating: ./zip_data/polyp/720501.jpg  
  inflating: ./zip_data/polyp/720601.jpg  
  inflating: ./zip_data/polyp/720650.jpg  
  inflating: ./zip_data/polyp/720950.jpg  
  inflating: ./zip_data/polyp/721001.jpg  
  inflating: ./zip_data/polyp/721020.jpg  
  inflating: ./zip_data/polyp/721050.jpg  
  inflating: ./zip_data/polyp/721101.jpg  
  inflating: ./zip_data/polyp/721120.jpg  
  inflating: ./zip_data/polyp/721150.jpg  
  inflating: ./zip_data/polyp/72120.jpg  
  inflating: ./zip_data/polyp/721220.jpg  
  inflating: ./zip_data/polyp/721250.jpg  
  inflating: ./zip_data/polyp/721301.jpg  
  inflating: ./zip_data/polyp/721350.jpg  
  inflating: ./zip_data/polyp/721450.jpg  
  inflating: ./zip_data/polyp/72150.jpg  
  inflating: ./zip_data/polyp/721501.jpg  
  inflating: ./zip_data/polyp/721520.jpg  
  inflating: ./zip_data/polyp/721550.jpg  
  inflating: ./zip_da

In [5]:

zip_dir = "./zip_data"
save_dir = "./drive/MyDrive/laundry_ckpoint"

json_dir = "./zip_data/polyj"
image_dir = "./zip_data/polyp"

json_files = []
image_files = []

for train_file in os.listdir(json_dir):
    json_files.append(os.path.join(json_dir, train_file))
    image_files.append(os.path.splitext(train_file)[0])

print("데이터 개수:", len(json_files))
print("이미지 개수:", len(image_files))
# print("json 장르", json_genre)
# print("image 장르", image_genre)

데이터 개수: 19215
이미지 개수: 19215


In [6]:
# os.mkdir(f"{save_dir}")

In [7]:
train_image, val_image, train_json, val_json = train_test_split(image_files, json_files, test_size=0.2, random_state=42, shuffle=True)
print(len(train_image), len(val_image))
print(len(train_json), len(val_json))

15372 3843
15372 3843


In [8]:
clothes_class = ["아우터", "하의", "원피스", "상의"]
clothes_xy = {"outer_xy":None, "pants_xy":None, "onepiece_xy":None, "shirt_xy":None}
clothes_fabric = {"outer_fabric":None, "pants_fabric":None, "onepiece_fabric":None, "shirt_fabric":None}

train_image_dict = {}
valid_image_dict = {}

for file in tqdm(train_json):
    if os.path.splitext(file.split("/")[-1])[0] not in train_image: #이미지 파일 없을 때
        continue
    with open(file, "rb") as f:
        data = json.load(f)
        file_id = data['이미지 정보']['이미지 식별자']
        file_name=str(data['이미지 정보']['이미지 식별자'])+'.jpg'
        for cls, xy, fabric in zip(clothes_class, clothes_xy, clothes_fabric):
            clothes_xy[xy] = data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][cls][0].values()
            if not clothes_xy[xy]:
                continue
#             if list(clothes_xy[xy])[2] < 1 or list(clothes_xy[xy])[3] < 1:
#                 print(file)
#                 continue
            try:
                clothes_fabric[fabric] = data['데이터셋 정보']['데이터셋 상세설명']['라벨링'][cls][0]['소재']
            except:
                pass
#                 print(file_name) #소재 없는 파일 
#                 print("--------------------")
        #print(clothes_xy)
        #print(clothes_fabric)

    for xy, fabric in zip(clothes_xy.values(), clothes_fabric.values()):
        if not xy:
            continue
        if list(xy)[2] < 1 or list(xy)[3] < 1:
            continue
        train_image_dict[(file_name, xy)] = fabric

        

for file in tqdm(val_json):
    if os.path.splitext(file.split("/")[-1])[0] not in val_image: #이미지 파일 없을 때
        continue
    with open(file, "rb") as f:
        data = json.load(f)
        file_name = str(data['이미지 정보']['이미지 식별자'])+'.jpg'
        for cls, xy, fabric in zip(clothes_class, clothes_xy, clothes_fabric):
            clothes_xy[xy] = data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][cls][0].values()
            if not clothes_xy[xy]:
                continue
            try:
                clothes_fabric[fabric] = data['데이터셋 정보']['데이터셋 상세설명']['라벨링'][cls][0]['소재']
            except:
                pass
        # print(clothes_xy)
        # print(clothes_fabric)

    for xy, fabric in zip(clothes_xy.values(), clothes_fabric.values()):
        if not xy:
            continue
        if list(xy)[2] < 1 or list(xy)[3] < 1:
            continue        
        valid_image_dict[(file_name, xy)] = fabric     
        
# train_image_dict
# valid_image_dict

100%|██████████| 3843/3843 [00:00<00:00, 6041.63it/s]


In [9]:
#사용자 정의 데이터셋
class ClothDataset(Dataset):
    def __init__(self, train_image_dict, data_dir=r"./drive/MyDrive/polyp"): #C:\Users\dra04\fashion image\K-Fashion 이미지\Training\원천데이터
        super().__init__()
        self.data_dir = data_dir
        self.train_image_dict = train_image_dict
        self.image_files = []
        self.dict_label_to_num = {
            "메시": 0,"데님" : 0,"면": 0 ,"합성섬유" : 0, "우븐": 0 ,"패딩" : 0,
            "니트":1,"레이스":1,"린넨":1,"플리스":1,"네오프렌":1, "저지":1,"울/캐시미어" :1,"헤어 니트": 1,
            "퍼":2,"앙고라":2,"코듀로이":2,"실크":2,"스판덱스":2,"트위드":2,"자카드":2 ,"벨벳":2,"시폰":2,
            "무스탕":3 ,"스웨이드":3,"시퀸/글리터":3,"가죽":3,"비닐/PVC":3
            
            # "퍼" : 2,
            # "니트" : 1,
            # "무스탕" : 3,
            # "레이스" : 1,
            # "스웨이드" : 3,
            # "린넨" : 1,
            # "앙고라" : 2,
            # "메시" : 0,
            # "코듀로이" : 2,
            # "플리스" : 1,
            # "시퀸/글리터" : 3,
            # "네오프렌" : 1,
            # "데님" : 0,
            # "실크" : 2,
            # "저지" : 1,
            # "스판덱스" : 2,
            # "트위드" : 2,
            # "자카드" : 2,
            # "벨벳" : 2,
            # "가죽" : 3,
            # "비닐/PVC" : 3,
            # "면" : 0,
            # "울/캐시미어" : 1,
            # "시폰" : 2,
            # "합성섬유" : 0,
            # "우븐" : 0,
            # "패딩" : 0,
            # "헤어 니트": 1
        } #0:물세탁, 1:울세탁, 2:드라이클리닝, 3:기타
        self.setup()

    def setup(self):
        for (file_name, xy), fabric in self.train_image_dict.items():
            if not fabric:
                continue
            self.image_files.append((os.path.join(self.data_dir, file_name), xy, fabric))

    def __getitem__(self, idx):
        file_name, xy, fabric = self.image_files[idx]
        fabric=fabric[0]
        x, y, w, h = map(int, xy)
        im = Image.open(file_name)
        im = transforms.ToTensor()(im)
        im = torchvision.transforms.functional.crop(im, y, x, h, w)
        im = transforms.Resize((300,300))(im)

        fabric_label = self.dict_label_to_num[fabric]
        return im, fabric_label
    
    def __len__(self):
        return len(self.image_files)



In [10]:
train_dataset = ClothDataset(train_image_dict)
valid_dataset = ClothDataset(valid_image_dict)
# print(train_dataset[0][0])
print(len(train_dataset))
print(len(valid_dataset))

22401
5651


In [11]:
import torch.nn.functional as F

class Resnet50(nn.Module): #18
    def __init__(self, num_classes):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained=True) #50

        self.fc = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return self.fc(x)


In [12]:
###########################
classes=["water",
        "wool",
        "dry cleaning",
        "etc"] 

# y=train_dataset[18][1] #dict의 18번째 label-->label 값을 뜻한다
figure = plt.figure(figsize=(8,8))
cols, rows= 3,3

for i in range(1, cols*rows+1):
    sample_idx=torch.randint(len(train_dataset),size=(1,)).item()
    
    img, label= train_dataset[sample_idx]
    
    figure.add_subplot(rows, cols, i)
    title=classes[label]
    plt.title(title)
    plt.axis("off")
    plt.imshow(img.permute(1,2,0))
plt.show()

# print(classes[2])
# predicted, actual = classes[preds[0].argmax(0)], classes[y]

In [15]:
BATCH_SIZE = 64 #32 작은 것보다 오히려 큰게 나을수도
EPOCHS =10
LOG_INTERVAL=35 #100
LEARNING_RATE=0.001 #0.001
# SAVE_DIR="./results"

device=torch.device("cuda") # if torch.cuda.is_available() else "cpu"

train_loader=DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True,shuffle=True)
valid_loader=DataLoader(valid_dataset, batch_size=BATCH_SIZE, drop_last=True,shuffle=True)

model = Resnet50(num_classes=4).cuda() #gpu
# model = Resnet50(num_classes=28).cuda() #gpu
# model = Resnet50(num_classes=28)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=0.1)
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

best_val_acc = 0
best_val_loss = np.inf

for epoch in range(EPOCHS):
    model.train()
    loss_value=0
    matches=0
    for idx, batch in enumerate(tqdm(train_loader)): #train_loader
        inputs, labels=batch
        
        inputs=inputs.to(device)
        labels=labels.to(device)
        
        optimizer.zero_grad()
        outs=model(inputs)
                
        preds=torch.argmax(outs,dim=-1)

        # num_label=labels.tolist()[0]     
        # num_pred=preds.tolist()[0]
        # plt.imshow(inputs[0].permute(1,2,0))
        # plt.show()
        # print(f"Label: {num_label}, Class: {classes[num_label]}, pred: {num_pred}, Class_pred: {classes[num_pred]}")

        loss=criterion(outs, labels)

        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds==labels).sum().item()
        if(idx+1) % LOG_INTERVAL==0:
            train_loss=loss_value / LOG_INTERVAL
            train_acc=matches / BATCH_SIZE / LOG_INTERVAL
            print(
                f"Epoch[{epoch}/{EPOCHS}]({idx + 1}/{len(train_loader)}) || " #train_loader
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%}"
            )
            torch.save(model.state_dict(), f"{save_dir}/{idx}.pth")
            loss_value = 0
            matches = 0
            
    with torch.no_grad():
        print("Calculating validation results...")
        model.eval() #검증 모드,검증 목적은 최적의 epoch를 찾아 overfiting과 unseen data 문제 를 막는 것
        val_loss_items=[]
        val_acc_items=[]

        for val_batch in tqdm(valid_loader):
            inputs, labels=val_batch
            inputs=inputs.to(device)
            labels=labels.to(device)

            outs=model(inputs)
            preds=torch.argmax(outs, dim=-1)

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

        val_loss= np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_dataset)
        best_val_loss = min(best_val_loss, val_loss) #검증용 데이터를 모델에 넣어 loss(MSE(used 손실함수))를 확인해 트레이닝이 잘 되고 있는지 확인(모델 사용)
        
        if val_acc > best_val_acc:
            print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            torch.save(model.state_dict(), f"{save_dir}/best.pth")
            best_val_acc = val_acc
        
        torch.save(model.state_dict(),f"{save_dir}/last.pth")
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
                  
    

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

 10%|▉         | 34/350 [12:36<1:58:08, 22.43s/it]

Epoch[0/10](35/350) || training loss 1.222 || training accuracy 54.29%


 20%|█▉        | 69/350 [24:32<1:29:18, 19.07s/it]

...

Epoch[9/10](350/350) || training loss 0.6794 || training accuracy 70.58%


100%|██████████| 350/350 [09:43<00:00,  1.67s/it]


Calculating validation results...


100%|██████████| 88/88 [01:48<00:00,  1.23s/it]


[Val] acc : 63.35%, loss: 0.85 || best acc : 67.03%, best loss: 0.75


In [14]:
torch.cuda.empty_cache()